In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# 사용자 코딩
# StackingClassifier 적용예정
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 2000)

# 결측치 해결.
X_id = X_test['cust_id']
X_temp = pd.concat([X_train, X_test], axis=0)
X_temp.fillna(0, inplace=True)

# 이상치 해결
# 컬럼을 이터레이션 하면서 사분위 편차를 적용하여 상한치를 변경 한다
print(X_temp.columns)
# 'cust_id', '총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수', '주말방문비율', '구매주기'
cols = ['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']

# import warnings
# warnings.filterwarnings("ignore")

for item in cols:
	print(item, " 항목 조정 중--")
	quan25 = X_temp[item].quantile(0.25)
	quan75 = X_temp[item].quantile(0.75)
	quan_diff = (quan75 - quan25) * 1.5
	mask = X_temp[item] > quan75 + quan_diff
	X_temp.loc[mask, item] = quan75 + quan_diff
	
# 변수 선택 및 데이터 분리
X_cat = X_temp[['주구매상품', '주구매지점']]
X_num = X_temp[['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']]

# 더미 변수화
X_cat_dummy = pd.get_dummies(X_cat)

from sklearn.preprocessing import MinMaxScaler
X_num_scaled = MinMaxScaler().fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, index=X_num.index, columns=X_num.columns)

# 데이터 병합 및 학습 테스트 데이터 분리
X_temp = pd.concat([X_num_scaled, X_cat_dummy], axis=1)
X_train = X_temp[:3500]
X_test = X_temp[3500:]
y_train = y_train['gender']

# 모델 적용 - 로지스틱, SVM, 랜덤 포레스트
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

estimators = [('sv', SVC()), ('rf', RandomForestClassifier())]
model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()).fit(X_train, y_train)
# print(model.score(X_train, y_train))

# 교차검증 진행
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# kfold = KFold(n_splits=3, shuffle=True, random_state=333)
# score = cross_val_score(StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()), X_train, y_train, cv=kfold)
# print("Cross Validation Score : >>", score.mean())

# Test 데이터 예측
pred_test = model.predict_proba(X_test)
result = pd.DataFrame({'cust_id': X_id, 'gender': pred_test[:, 1]})
result.to_csv('003000123.csv', index=False)

import os.path
print("exists file ? : ", os.path.exists('003000123.csv'))
# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)


궁금한점

- StackingClassifier 외 VotingClassifier 도 predict_proba 가 가능한지?
- predict_proba가 분류기 특성인지 아니면 마지막 알고리즘인 LogisticRegression 이라서 지원 된 건지?


- 이상치 평활화를 했는데 하는데 더 나은 건지?!?
  - 평활화 미적용 : Cross Validation Score : >> 0.6497141468524308
  - 평활화 적용후 : Cross Validation Score : >> 0.6542857394824219 (적용하는 쪽이 조금이나마 더 나음)